In [1]:
# Load the aiida profile
from aiida import load_profile
aiida_profile = load_profile()
aiida_profile.name


'default'

In [ ]:
## INFO:: In this class all the tools for bunch of wc submission have been created 

class DataUtils:

## SECTION1-1 : In this section to delete the remote workdir for calcjob and  nodes from the DB     
    # please note that it is tested for one calc list
    def delete_remote_workdir(self, pks: list, verbosity=0, dry_run= True):
        from aiida.common import exceptions
        from aiida.orm import load_node
        from aiida.orm import computers
        import sys
        # TODO : add the verbosity as discused here 
        # https://aiida.readthedocs.io/projects/aiida-core/en/v1.5.0/_modules/aiida/manage/database/delete/nodes.html

        """
        :param pks: calc node list

        """
        removed_path_list = [] # The part of the path will be deleted
        remote_path_list = []  # The original path
        updated_path_list = [] # After removing the part of the path
        loadable_list = [] # To load the node and save it loadable_list
        loaded_node_list = []
        # To check the loadable calcjob list
        for pk in pks:
            try:
                loaded_node = load_node(pk)
            except exceptions.NotExistent:
                print('This is calcjob node'.format(pk))
                loaded_node = pk
                loaded_node_list.append(loaded_node)
                #             sys.exit()
            else:
                loaded_node_list.append(loaded_node)
        # Computer data

        for node in loaded_node_list:
            load_pk = node
            # computer data
            computer = load_pk.computer
            computer_name = computer.label
            print(computer_name)

            remote_path = load_pk.get_remote_workdir()
            remote_path_list.append(remote_path)

            delete_folder = remote_path.split('/')[-1]
            removed_path_list.append(delete_folder)

            new_remote_path = remote_path.replace(remote_path.split('/')[-1], '')
            updated_path_list.append(new_remote_path)

        if dry_run or verbosity == 3:

            for i, paths in enumerate(zip(remote_path_list, updated_path_list)):
                print('Before the delation the original path list : {}\n'.format(paths[0]))
                print('After deletion the modefied or updated path : {}'.format(paths[1]))
        if verbosity == 3 or verbosity == 2:
            val = input("Are you agree to clean the remote workdir (y/n) : ")
        else:
            val = 'y'
        if str(val)=='y' or str(val)=='Y':
            if not dry_run:
                for remote_path in remote_path_list:

                        # Open the connection to the remote folder/dir via transport
                        computer_transport = computer.get_transport()
                        is_transport_open = computer_transport.is_open
                        if not is_transport_open:
                            computer_transport.open()
                        try:
                            computer_transport.rmtree(remote_path)
                        except IOError as ex:
                            print(ex)
        else:
            print('Nothing to clean from the remote workdir!')
    ## It returns all the calcjob from a WC node
    def find_calcJob(self, pk_or_node, debug=True):

        calcjob_node_list=[]
        wc_node_list = []
        try:
            if isinstance( pk_or_node, int):
                if debug:
                    print('This is pk')
                node = load_node(pk_or_node)
            else:
                if debug:
                    print('This is node.')
                node= pk_or_node
        except:
            print('{} is nither node ID nor aiida_node. '.format(pk_or_node))

        ## Use the get_calcjob_wc to get descendent calcjob list and  wc list
        calc_list, wc_list = get_calcjob_wc(node)
        calcjob_node_list += calc_list

        while len(wc_list)!=0:
            new_wc_list = []

            for i in wc_list[:]:
                calc_list, wc_list = get_calcjob_wc(i)
                new_wc_list += wc_list
                calcjob_node_list += calc_list

            wc_list = new_wc_list

        return calcjob_node_list

    ## This function returns calcjob_list and wc_list from a wc or calcjob node   
    def get_calcjob_wc(self, node):
        """
        :param: node
        :return: workchain node list and calcjob node list
        """ 
        from aiida.orm import CalcJobNode, WorkChainNode
        wc = []
        calc_job = []

        if node.node_type == 'process.workflow.workchain.WorkChainNode.':

        # here all outgoing worchain node
            out_going_wc = node.get_outgoing(node_class=WorkChainNode).all()
            wc = [i.node for i in out_going_wc[:]]

        # here all outgoing calcjob node
            out_going_calc = node.get_outgoing(node_class=CalcJobNode).all()
            calc_job = [i.node for i in out_going_calc[:]]

        elif node.node_type == 'process.calculation.calcjob.CalcJobNode.':
            calc_job.append(node)

        return calc_job, wc
    
    # This is the final del_node_function. Using this function for any specific wc node the node from the 
    # Db as well as the calcjob data from the remote workdir can be deleted.
    def del_node(self, node_pks, dry_run=True, verbosity=3, debug=True, only_remote_dir=False,
                only_database=False):
        """
        1. This function will delete the node data from the database and also from the remote_dir

        :params node_pks: (list) list of workchain to delete from database as well as from remote workdir
        :param verbosity: 0 prints nothing.  This is for workdir and wc
                          1 prints just sums and total.   This is for workdir but not for wc
                          2 prints indivisual nodes.  This is for workdir and wc
        :param dry_run: Do not delete anything just show the status as in the verbosity given
        """
        from aiida.orm import load_node
        from aiida.manage.database.delete.nodes import delete_nodes

        calcjobs_list = []
        pks_given = []
        for i in node_pks:    
            try:
                if isinstance( i, int):
                    if debug:
                        print('This might be pk or uiid')
                    node = load_node(i)
                else:
                    if debug:
                        print('This might be a node.')
                    node= i
            except:
                print('{} is nither node ID nor aiida_node. '.format(i))

            pks_given.append(node.pk)

            calcjobs = find_calcJob(node, debug)
            calcjobs_list += calcjobs
            print('calcjob list : ', calcjobs_list,)
        if only_remote_dir:
            delete_remote_workdir(calcjobs_list, verbosity=verbosity, dry_run=dry_run)
        if only_database:
            delete_nodes(pks_given, verbosity=verbosity, dry_run=dry_run,force=False)

## SECTION-1: Ends here

## SECTION-2: In this section group creation, check wc in group, save wc in group

    def group_not_exist_create(self, group_label, group_descr=None):
        from aiida.orm import load_group, Group
        """
            Check the group exist either must create
        """
        if group_descr==None:
            group_descr='No Description is added'

        try:
             group = load_group(group_label)
        except:
            print('Group named {} is not exist but is being created .'.format(group_label))
            group = Group(label=group_label, description=group_descr)
            group.store()
            print('Newly created group pk : {}'.format(group.pk))
        return group


    def check_wc_exist_in_group(self, group, wc_label=None, wc_pk=None, wc_node= None):
        if not isinstance(group, Group):
            try:
                group= load_group(group)
            except:
                return(f'Exception ocours while loading group ID {group}')
            
        nodes_list = list(group.nodes)
        nodes_label = [i.label for i in nodes_list[:]]
        nodes_pk = [i.pk for i in nodes_list[:]]

        if wc_label in nodes_label:
            print('node_label : {} is exist in the group : {}.'.format(wc_label, group.label))
            return True

        elif wc_pk in nodes_pk:
            print('node_pk-{} is exist in the group-{}.'.format(wc_pk, group.label))
            return True
        
        elif wc_node in nodes_list:
            print('node-{} is exist in the group-{}.'.format(wc_node, group.label))
            return True
        
        else:
            return False
    
    def create_group_save_wcs(self,group_label: str="", group_descr: str="", verbose: bool=False,
            wc_list: list = [], wc_dict: dict = {}, debug: bool = True
                            ) -> Group :
        from aiida.orm import load_group, load_node, Node
        " A new group named as <group_label> wil be created if not exist is db and wc from <wc_list> or <wc_dict> will be store.
          wc_list: consists pk
          wc_dict: consists keys node pk, label, description
          Return the actual group
        "


        group = self.group_not_exist_create(group_label=group_label, group_descr=group_descr)
        members_list = group.nodes
        members_label = [i.label for i in nodes_list[:]]
        members_pk = [i.pk for i in nodes_list[:]]
        
        if wc_list != []:
            for ID in wc_list[:]:
                if isinstance(ID, Node):
                    node = ID
                else:
                    try:
                        node = load_node(ID)
                    except:
                        print(f'This is not node neither identifire given as {ID})
                if node.label not in members_label:
                    if node.pk not in members_pk:
                        group.add_nodes(ID)
                else:
                    val = input(f"Node-{ID} is already exist in group-{group.pk}. Do you want to store this node in the mentioned group? (Y/N)")
                    if val == 'y' or val == 'Y':
                        group.add_nodes(ID)

        elif wc_dict != {}:
            for i in wc_dict.keys():
                node = load_node(wc_dict[i]['pk'])
                if node.label not in members_label:
                    if node.pk not in members_pk:
                        group.add_nodes(node)
                else:
                    val = input(f"Node-{node} is already exist in group-{group.pk}. Do you want to store this node in the mentioned group? (Y/N)")
                    if val == 'y' or val == 'Y':
                        group.add_nodes(ID)

        return group


## SECTION-2: Ends here.



In [4]:
class single_submit:
    from aiida.engine import submit
## SECTION-1 : In this secction combine imp
    ## In this method  combine imp from two preconverged single imp_wc.
    def submit_combine_imp(self, imp1_wc_node, imp2_wc_node,  offset_imp2, kkr_code=None, kkr_imp_code= None, 
                           settings= None, dry_run= True, label= None, gf_host_remote=None, options= None, 
                           scf_wf_parameters = None, params_kkr_overwrite=None):
        
        from aiida_kkr.workflows import combine_imps_wc, kkr_imp_sub_wc, kkr_flex_wc
        from aiida.orm import Dict
        imp1_output = imp1_wc_node.outputs.workflow_info
        imp2_output = imp2_wc_node.outputs.workflow_info
        
        if scf_wf_parameters==None:
            sub_wc1_node = imp1_wc_node.get_outgoing(node_class=kkr_imp_sub_wc).first().node
            scf_wf_parameters = sub_wc1_node.inputs.wf_parameters
        
        if gf_host_remote != None:
            if params_kkr_overwrite==None:
                sub_gf_write_out = imp1_wc_node.get_outgoing(node_class=kkr_flex_wc).first().node
                params_kkr_overwrite = sub_gf_write_out.inputs.params_kkr_overwrite
        if settings==None:
            settings = imp1_wc_node.inputs.wf_parameters_overwrite
        if label==None:
            label = 'pk' + str(imp1_wc_node.pk)+':'+ str(imp1_wc_node.pk)
        
        if kkr_code == None:
            kkr_code= imp1_wc_node.inputs.kkr
        if kkr_imp_code == None:
            kkr_imp_code=  imp1_wc_node.inputs.kkrimp
        if options == None:
            options = imp1_wc_node.inputs.options       

        builder = combine_imps_wc.get_builder()
        builder.impurity1_output_node = imp1_output
        builder.impurity2_output_node = imp2_output
        if isinstance(offset_imp2, dict):
            builder.offset_imp2 = Dict(dict=offset_imp2)
        else:
            builder.offset_imp2 = offset_imp2

        builder.scf.kkrimp = kkr_imp_code
        builder.scf.options = options
        builder.scf.wf_parameters = scf_wf_parameters

        if gf_host_remote==None:
            builder.host_gf.kkr = kkr_code
            builder.host_gf.options = options
            builder.host_gf.params_kkr_overwrite = params_kkr_overwrite #host_gf.inputs.wf_parameters
        else:
            builder.gf_host_remote = gf_host_remote
        if settings!=None:
            if isinstance(settings, dict):
                builder.wf_parameters_overwrite = Dict(dict=settings)
            else:
                builder.wf_parameters_overwrite = settings

        builder.metadata.label = label
        if not dry_run:
            submission = submit(builder)
            return submission
        else:
            msg = ' This is dry_run. '
            return msg


## SECTION-1 : Ends here

## SECTION-2 : In this section single imp dos will be prepared
    ##  dos wc from the pre converged kkr_imp_wc
    def single_imp_dos_from_scf(self, preconverged_kkr_imp_node, kkr= None, kkrimp= None, options=None,
                                wf_parameters=None, host_remote= None, gf_dos_remote= None, kkrimp_remote=None,
                                imp_pot_sfd= None, impurity_info= None, params_kkr_overwrite= None,
                                dry_run= True
                               ):
        
        from aiida_kkr.workflows import kkr_imp_dos_wc
        builder= kkr_imp_dos_wc.get_builder()
        if kkr == None:
            builder.kkr= preconverged_kkr_imp_node.inputs.kkr
        if kkrimp == None:
            builder.kkrimp= preconverged_kkr_imp_node.inputs.kkrimp
        if options == None:
            builder.options= preconverged_kkr_imp_node.inputs.options
        if wf_parameters != None:
            builder.wf_parameters= wf_parameters
        if host_remote != None:
            builder.host_remote = host_remote
        if gf_dos_remote != None:
            builder.gf_dos_remote = gf_dos_remote
        if kkrimp_remote != None:
            builder.kkrimp_remote = kkrimp_remote
        if imp_pot_sfd != None:
            builder.imp_pot_sfd = imp_pot_sfd
        if impurity_info != None:
            builder.impurity_info = impurity_info
        if params_kkr_overwrite != None:
            builder.params_kkr_overwrite != params_kkr_overwrite
        if dry_run:
            return(f'This is dryrun for single_imp_dos_from_scf()')
        elif not dry_run:
            submission = submit(builder)
    
    
    def single_imp_dos_restart(self, pre_kkr_imp_dos, wf_parameters, params_kkr_overwrite= None dry_run = True):
        
        from aiida.engine import submit
        builder= pre_kkr_imp_dos.get_builder_restart()
        builder.wf_parameters= wf_parameters
        builder.params_kkr_overwrite = params_kkr_overwrite
        if dry_run:
            return(f'This is dry run for single_imp_dos_restart')
        elif not dry_run:
            submission = submit(builder)
            return submission
        
        
        
        
        

## SECTION-2 : Ends Here

In [ ]:
class bunch_wc:
    """
        In this section bunch of wc will be submitted.
    """

    def __init__(self):
        self.Max_fail = 3
        self.Max_submit = 30
        self.Is_sumit_finished = False
from aiida.orm import Code, Dict, RemoteData
from typing import Union
## --------------------------------------------
## SECTION-1: This is the part for prepareing of bunch of calculation
# and this part will consist also dict of bunch
    @classmethod
    def combine_imps_bunch_settings(cls, si_imp_list1: list, si_imp_list2: list, kkr_code: Code= None, kkr_imp_code: Code=None,
                          builder_options: Dict = None, succ_group_label: str: None, succ_group_descr: str = None,
                          offset_imp2:Union[dict, Dict] = {'index':1}, max_submission: int=30, settings: Union[dict, Dict] =None,
                          gf_host_remote: RemoteData= None, scf_wf_parameters: Union[dict, Dict]=None, debug: bool= False,
                          params_kkr_overwrite: Union[dict,Dict]= None, dry_run: bool= True, max_fail_wc: int= 3
                          ):
        "
            In this method set all the inputs as function parameters from the user.
        "

        #TODO: Change all the si_imp_list1 into si_imp1_list
        #TODO: 
        from Bunch_wc_submission_package.submission_utils import submission_utils as su
        self = cls()
        self.Si_imp1_list= si_imp_list1
        self.Si_imp2_list= si_imp_list2
        self.Kkr_code = kkr_code
        self.Kkr_imp_code = Kkr_imp_code
        self.Options = builder_options

        # TODO: Think about codes kkr, kkr_imp, builder_options
        if succ_group_label == None:
            self.Succ_group_label= 'No_group_label_is_found'
        else:
            self.Succ_group_label= succ_group_label

        if succ_group_descr == None:
            self.Succ_group_desc= 'No_group_description_is_found'
        else:
            self.Succ_group_desc= succ_group_descr

        if fail_group_label == None:
            self.Fail_group_label= 'No_group_label_is_found_failed'
        else:
            self.Fail_group_label= str(succ_group_label) + '_failed'

        if fail_group_descr == None:
            self.Fail_group_desc= 'No_group_description_is_found_failed'
        else:
            self.Fail_group_desc= str(succ_group_descr) + '_failed'


        if isinstance(offset_imp2, dict):
            self.Offset_imp2= Dict(dict=offset_imp2)
        else:
            self.Offset_imp2= offset_imp2

        self.Max_submit= max_submission

        if isinstance(settings, dict):
            self.Settings = Dict(dict=settings)
        else:
            self.Settings = settings

        # TODO: If remote host gf is no given after the first the calc take it as from the first. Employ it later
        self.Gf_host_remote= gf_host_remote

        if isinstance(scf_wf_parameters, dict):
            self.Scf_wf_parameters= Dict(dict= scf_wf_parameters)
        else:
            self.Scf_wf_parameters= scf_wf_parameters

        self.Debug = debug

        if isinstance(params_kkr_overwrite, dict):
            self.Params_kkr_overwrite= Dict(dict=params_kkr_overwrite)
        else:
            self.Params_kkr_overwrite = params_kkr_overwrite

        self.Dry_run= dry_run

        self.Max_fail= max_fail_wc
        self.Si_submit = su.submit_combine_imp #The submit function

        self.All_combination_dict = su.create_combine_imps_combination(self.Si_imp1_list, self.Si_imp2_list)
        return self
    
    def create_combine_imps_combination(self, imp_list1: list, imp_list2: list)-> dict :
        "
            imp1_list1; imp2_list2 are the wc node lists.
            Create all possible combinations from two given lists.
            Returns dict consists of dict <wc_num> consist of <label>, and empty <submission node>. 
            The label for each combine_imps_wc is also create here.
        "
        if not isinstance(si_imp_list1, list):
            print('The given impurity wc list 1 is not the list type.')
            return print('Please provide single_imp1_list.')
        if not isinstance(si_imp_list2, list):
            print('The given impurity wc list 2 is not the list type.')
            return print('Please provide single_imp2_list.')

        for i in si_imp_list1[:]:
            for j in si_imp_list2[:]:
                node_truple = (i, j)
                pk_truple = (i.pk, j.pk)
                imp1_info = i.inputs.impurity_info.get_dict()
                imp2_info = j.inputs.impurity_info.get_dict()
                ilayer1 = str(imp1_info['ilayer_center'])
                ilayer2 = str(imp2_info['ilayer_center'])
                try:
                    label= i.label.split(':')[0] + ':'+ j.label+'_il_'+ilayer1+'_il_'+ilayer2
                else:
                    #print(f"INFO: Single_imp_wc does not have any label ")
                    label = f"pk:{i.pk}:{j.pk}"

                all_combination_dict[tot_wc_num] = {'label' : label,
                                                    'submission': None}
        return all_combination_dict


    
## SECTIION-"": In this section general submit will be preapared to submit any of the bunch wc with 
    # coresponding dict of bunch

    def Submit(self):

        from Bunch_wc_submission_package.submission_utils import submission_utils as su

        # TODO: maybe 'part_1' can be added with the for loop summittion 
        ## Call required tools from the submission utils
        all_combination_dict = self.All_combination_dict.copy()
        all_resedue_dict = all_combination_dict.copy()# resedu dict for all the failed dict
        all_success_dict = {}
        all_submission_dict = {} #TODO change the all_submission dict as all running dict 
        all_failed_dict = {}
        ## some integer variables to track the numbers
        tot_wc_num = len(all_combination_dict)
        all_submission_num = 0
        all_success_num = 0
        all_failed_num = 0
        ## part_1---

        succ_group = su.group_not_exist_create(label = self.Succ_group_label, description= self.Succ_group_descr)
        fail_group = su.group_not_exist_create(label = self.Fail_group_label, description= self.Fail_group_descr)

        for key, val in all_combination_dict.items():

            label = all_combination_dict[key]['label']

            wc_pre_exist = su.wc_lbl_in_group(group=succ_group, node_label=label)
            if wc_pre_exist:
                print('Already one wc named as {} is exist in this group'.format(label))
                continue
         # TODO: In the docs add the keys in the all_combination_dict[dict_label]'submission' key as empty and 'label' key for each ubmited wc also mention that key in all_combination_dict[key] is for labeling and tracking all the combine dict
            submission = su.submit_combine_imp(key_value = val, obj=self)
            print(f"THe submitted combine Imps is : {submission.pk}")
            all_submission_dict[key] = all_combination_dict[key].copy()
            all_submission_dict[key]['submission'] = submission
            all_submission_num += 1
            while((all_submission_num - all_success_num - all_failed_num >= max_submission) or
                    (tot_wc_num - all_submission_num == 0) or (all_failed_num == max_fail_wc)):
                    t.sleep(60*2)

    #       --------------------------------------------------------------------------------
                    pop_list = []
                    for submit_key in all_submission_dict.keys():
                        submission = all_submission_dict[submit_key]['submission']
                        if submission.is_finished == True:
                            if submission.exit_status == 0 :
                                all_success_num +=1
                                if all_success_num%5==0:
                                    print('all_success_num : ', all_success_num)
                                all_success_dict[submit_key] = all_combination_dict[submit_key].copy()
                                all_resedue_dict.pop(submit_key)

                                succ_group.add_nodes(submission)
                                pop_list.append(submit_key)
                                su.del_node(node_pks=[submission.pk], dry_run=False, verbosity=0, debug=False, only_remote_dir=True)
                            else:
                                all_failed_num += 1
                                all_failed_dict[submit_key] = all_combination_dict[submit_key].copy()

                                print('INFO: all_failed_num : ', all_failed_num)
                                fail_group.add_nodes(submission)
                                pop_list.append(submit_key)

                        elif submission.is_excepted:
                            all_failed_num += 1
                            all_failed_dict[submit_key] = all_combination_dict[submit_key].copy()
                            print('all_failed_num : ', all_failed_num)
                            fail_group.add_nodes(submission)
                            pop_list.append(submit_key)

    #           -------------------------------------------------------------------------------
                    garbage = [all_submission_dict.pop(pop_key) for pop_key in pop_list[:]]

                    # To break while loop here
                    if all_failed_num == max_fail_wc:
                            if (all_submission_num - all_success_num + all_failed_num) == 0 :
                                break
            # To break for loop here
            if all_failed_num == max_fail_wc:
                if (all_submission_num - all_success_num + all_failed_num) == 0:
                    break
        ## To save the success dict in the Bunch_wc 
        self.All_success_dict = all_success_dict.copy()
        self.All_resedue_dict = all_resedue_dict.copy()
        self.All_failed_dict = all_failed_dict.copy()
        self.Is_sumit_finished = True


    def ReSubmit(self):
        if self.Is_sumit_finished:
            resedu_dict = sef.All_resedue_dict
            self.All_combination_dict = resedu_dict.copy()
            self.submit()
## --------------------------------------------------------